## Here we demostrate why naive Bayes seems to fail for this data set.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.insert(0,'../libs/')
import data_cleaning as dc

from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score

/usr/local/lib/python3.5/dist-packages/matplotlib/backends/backend_gtk3agg.py:18: UserWarning: The Gtk3Agg backend is known to not work on Python 3.x with pycairo. Try installing cairocffi.
  "The Gtk3Agg backend is known to not work on Python 3.x with pycairo. "


In [2]:
df = pd.read_csv("../data/train.csv")
df = dc.classify_colors(df)
df = dc.massage_df(df)

In [3]:
df_dog = df[df.AnimalType=='Dog'].copy(deep=True)
df_cat = df[df.AnimalType=='Cat'].copy(deep=True)

dc.fix_cat_breed(df_cat)

Start by fitting all the features we thought were useful.

In [4]:
# Selects the importand 
cat_features = ['age_numeric', 'sex', 'mixed', 'catbreed','day_of_week','month','neuter_status']
colors = ['has_Blue', 'has_Tortie',
       'has_White', 'has_Brown', 'has_Lilac', 'has_Point', 'has_Tabby',
       'has_Gray', 'has_Agouti', 'has_Black', 'has_Lynx', 'has_Orange',
       'has_Tan', 'has_Calico']

df_cat_dummies = pd.get_dummies(df_cat[cat_features + colors].dropna())
df_cat_dummies

# splits data.
rows = np.random.choice(df_cat_dummies.index.values
                ,int( round(len(df_cat_dummies)*.6) ), replace=False)
df_cat_dummies_train = df_cat_dummies.ix[rows]
df_cat_dummies_test = df_cat_dummies.drop(rows)

#### fit and test model

train_features = df_cat_dummies_train
train_outcomes = df_cat.ix[df_cat_dummies_train.index]['OutcomeType']
test_features  = df_cat_dummies_test
test_outcomes  = df_cat.ix[df_cat_dummies_test.index]['OutcomeType']

bnb = BernoulliNB()
bnb.fit(train_features,train_outcomes)

predictions = bnb.predict(test_features)
accuracy_score(test_outcomes,predictions)

0.72903080728581071

In [5]:
pd.Series(predictions).value_counts()

Adoption           2393
Transfer           2047
Return_to_owner       7
dtype: int64

In [6]:
df_cat['OutcomeType'].value_counts()

Transfer           5505
Adoption           4272
Euthanasia          710
Return_to_owner     500
Died                147
Name: OutcomeType, dtype: int64

So it's missing lots of Euthanasia. It mostly just predicts transfer vs. adoption.
Let's try only neuter status:

In [7]:
# Selects the importand 
cat_features = ['neuter_status']

# split into test and train
frac = .6 # use 66% data for training
test_rows = np.random.choice(df_cat.index.values
            ,int(round( len(df_cat)*frac )),replace=False )

# train/test features
dummies = pd.get_dummies(df_cat[cat_features])
mytrain = dummies.ix[test_rows]
mytest  = dummies.drop(test_rows)

# train/test outcomes
train_outcome = df_cat.ix[mytrain.index].OutcomeType
test_outcome  = df_cat.ix[mytest.index].OutcomeType

classifier = BernoulliNB()
classifier.fit(mytrain, train_outcome )

predictions = classifier.predict(mytest)
accuracy_score(df_cat.ix[mytest.index].OutcomeType,predictions)

0.74449932644813654

In [8]:
pd.Series(predictions).value_counts()

Adoption    2422
Transfer    2032
dtype: int64

Performs equally well! Seems like it's entirely based on neuter status, which works well to destinguish transfers and adoptions, likely because animals are transfered to facilities that can neuter the animal.

In [9]:
df_cat.loc[(df_cat['neuter_status']=='Intact')&(df_cat['OutcomeType']=="Adoption"),['neuter_status','OutcomeType']].count()

neuter_status    238
OutcomeType      238
dtype: int64

In [10]:
df_cat.loc[(df_cat['neuter_status']!='Intact')&(df_cat['OutcomeType']=="Adoption"),['neuter_status','OutcomeType']].count()

neuter_status    4034
OutcomeType      4034
dtype: int64

In [11]:
df_cat.loc[(df_cat['neuter_status']=='Intact')&(df_cat['OutcomeType']=="Transfer"),['neuter_status','OutcomeType']].count()

neuter_status    3234
OutcomeType      3234
dtype: int64

In [12]:
df_cat.loc[(df_cat['neuter_status']!='Intact')&(df_cat['OutcomeType']=="Transfer"),['neuter_status','OutcomeType']].count()

neuter_status    2271
OutcomeType      2271
dtype: int64

Out guess is that naive Bayes needs more data to predict things better. Neuter status is so stongly correlated with transfers that it can use this indicator, but cannot use any others.